In [42]:
import pdfplumber
import re

The following code block will scrape all text from the parcel report PDF -- which was generated from scraping KGIS.

In [43]:
pdf = pdfplumber.open("Parcel_Report.pdf")
all_pages = pdf.pages
cleaned_lines = []
cleaned_list_of_dicts = []
for each_page in all_pages:
    text_of_each_page = each_page.extract_text()
    all_lines = text_of_each_page.splitlines()
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
each_dict = {}
for index, cleaned_line in enumerate(cleaned_lines):
    if re.match(r"^09[0-9A-Z]{5,}$", cleaned_line):
        # parcel_id
        each_dict["parcel_id"] = cleaned_line
        each_dict["name"] = cleaned_lines[index + 1]
        cleaned_list_of_dicts.append(each_dict)
        each_dict = {}
    else:
        continue
    # if index % 2 == 0:
    # else:
# print(cleaned_list_of_dicts)


In [44]:
import pandas as pd

In [45]:
pd.set_option("display.max_rows", None)


In [ ]:
# df = pd.DataFrame(cleaned_list_of_dicts)
# df


In [46]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [51]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [/Users/areena.arora/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache
/var/folders/81/2s24wkdd02j9528vnjsx7m080000gn/T/ipykernel_89673/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [52]:
def get_all_data(parcel_number, agree):
    driver.get('https://propertyinfo.knoxcountytn.gov/search/commonsearch.aspx?mode=realprop')
    if agree:
        driver.find_element(By.ID, "btAgree").click()
    driver.find_element(By.ID, "inpParid").send_keys(parcel_number)
    driver.find_element(By.ID, "btSearch").click()
    error_passing = driver.find_elements(By.ID, "searchResults")
    print(error_passing)
    if len(error_passing) > 0:
        error_passing[0].find_element(By.CLASS_NAME, "SearchResults").click()
    driver.find_element(By.XPATH, '//*[@id="sidemenu"]/ul/li[3]/a').click()
    parcel_id_dirty = driver.find_element(By.CLASS_NAME, "DataletHeaderTop").text
    parcel_id = parcel_id_dirty.split(": ")[1]

    sales_details = driver.find_element(By.ID, "Sales Summary")
    rows = sales_details.find_elements(By.TAG_NAME, "tr")[1:]
    sales_data = []
    for row in rows:
        all_data = row.find_elements(By.TAG_NAME, "td")
        if len(all_data) < 5:
            print("insufficient data")
            continue
        else:
            data = {}
            date = all_data[0].text
            book = all_data[1].text
            page = all_data[2].text
            price = all_data[3].text
            grantee = all_data[-1].text
            data["date"] = date
            data["book_number"] = book
            data["page"] = page
            data["sale_price"] = price
            data["grantee_name"] = grantee
            data["parcel_id_new"] = parcel_id
            sales_data.append(data)
    return(sales_data)

In [49]:
# get_all_data("094LF015", True)

[<selenium.webdriver.remote.webelement.WebElement (session="0f9881b9bfe364f662362ad419bb698b", element="a3b365a4-bfd6-43c0-adaa-2523a859fb3a")>]
insufficient data


[{'date': '01/03/2005',
  'book_number': '20080225',
  'page': '0063382',
  'sale_price': '$0',
  'grantee_name': 'DEWHIRST DAVID W & MICHAEL E KENNEDY',
  'parcel_id_new': '094LF015'},
 {'date': '08/06/1998',
  'book_number': '00002295',
  'page': '0000916',
  'sale_price': '$76,000',
  'grantee_name': 'DEWHIRST DAVID W & MICHAEL E KENNEDY',
  'parcel_id_new': '094LF015'},
 {'date': '06/02/1988',
  'book_number': '00001956',
  'page': '0000029',
  'sale_price': '$81,000',
  'grantee_name': 'REICH ESTHER S',
  'parcel_id_new': '094LF015'}]

In [53]:
pd.set_option('display.max_columns', None)

all_sales_data = []
agree = True
for each_parcel_dict in cleaned_list_of_dicts[:25]:
    sales_data = get_all_data(each_parcel_dict["parcel_id"], agree)
    agree = False
    all_sales_data.extend(sales_data)
full_df = pd.DataFrame(all_sales_data)
full_df.head(10)

[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[<selenium.webdriver.remote.webelement.WebElement (session="11759785985a0c572a2654b8560cfec8", element="0846d9a6-ba1b-421f-bb40-44ab2c86604a")>]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[]
insufficient data
[<selenium.webdriver.remote.webelement.WebElement (session="11759785985a0c572a2654b8560cfec8", element="e05d0338-a647-4adf-badd-44bf0a1096a4")>]
insufficient data
[<selenium.webdriver.remote.webelement.WebElement (session="11759785985a0c572a2654b8560cfec8", element="98a3d7ac-a92f-4227-b670-b19e5a3e603a")>]
insufficient data
[]
insufficient data


,date,book_number,page,sale_price,grantee_name,parcel_id_new
0,08/16/1990,00002017,0000594,$0,PARKEY SYLVIA ROCHELLE,095GP014
1,07/14/1989,00001982,0001091,"$32,000",PARKEY SYLVIA ROCHELLE,095GP014
2,11/30/1994,00002160,0001032,"$20,686",LEE MARY N,095GN008
3,06/26/1973,00001521,0000692,$0,KNOXVILLES COMMUNITY DEV CORP,095GN008
4,10/18/1996,00002229,0000419,$0,BROWN MARTHA S COBB & JERVIS S,095GP010
5,09/29/1988,00001959,0001160,$0,BROWN MARTHA S COBB & JERVIS S,095GP010
6,12/08/1998,00002309,0000728,"$450,000",HEURISTIC WORKSHOP INC,094EF043
7,10/10/2000,20001115,0033622,"$29,200",LEVITT JOSEPH J JR,095BF027
8,01/19/2000,20000119,0003887,$0,RADER LILLIE,095BF027
9,08/24/1990,00002018,0000178,"$17,480",JOHNSON HATTIE,095BF027


In [54]:
full_df

,date,book_number,page,sale_price,grantee_name,parcel_id_new
0,08/16/1990,00002017,0000594,$0,PARKEY SYLVIA ROCHELLE,095GP014
1,07/14/1989,00001982,0001091,"$32,000",PARKEY SYLVIA ROCHELLE,095GP014
2,11/30/1994,00002160,0001032,"$20,686",LEE MARY N,095GN008
3,06/26/1973,00001521,0000692,$0,KNOXVILLES COMMUNITY DEV CORP,095GN008
4,10/18/1996,00002229,0000419,$0,BROWN MARTHA S COBB & JERVIS S,095GP010
5,09/29/1988,00001959,0001160,$0,BROWN MARTHA S COBB & JERVIS S,095GP010
6,12/08/1998,00002309,0000728,"$450,000",HEURISTIC WORKSHOP INC,094EF043
7,10/10/2000,20001115,0033622,"$29,200",LEVITT JOSEPH J JR,095BF027
8,01/19/2000,20000119,0003887,$0,RADER LILLIE,095BF027
9,08/24/1990,00002018,0000178,"$17,480",JOHNSON HATTIE,095BF027
